In [1]:
!pip install spacy
!python -m spacy download pt_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 1.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import spacy

nlp = spacy.load("pt_core_news_lg")

LEGAL_ENTITY_TYPES = {
    "PER",   # Pessoas
    "ORG",   # Órgãos, tribunais
    "LOC",   # Localização
    "DATE",  # Datas
    "MONEY"  # Valores monetários
}

def extract_entities(text):
    doc = nlp(text)
    return [
        ent.text.strip().lower()
        for ent in doc.ents
        if ent.label_ in LEGAL_ENTITY_TYPES
    ]


In [3]:
def compute_neprec(reference_text, generated_text):
    ref_entities = set(extract_entities(reference_text))
    gen_entities = extract_entities(generated_text)

    if len(gen_entities) == 0:
        return 1.0  # No hallucination if no entities used

    correct = sum(1 for ent in gen_entities if ent in ref_entities)
    precision = correct / len(gen_entities)
    return precision


In [4]:
import os
import pandas as pd

file_path = 'hallucination_analysis.xlsx'

if not os.path.exists(file_path):
    print(f"ERRO: O arquivo {file_path} não foi encontrado. Por favor, faça o upload no menu lateral esquerdo.")
else:
    # 1. Carregar o dataset
    df = pd.read_excel(file_path)
    print(f"Dataset carregado. Total de linhas: {len(df)}")

Dataset carregado. Total de linhas: 210


In [5]:
df["neprec"] = df.apply(
    lambda row: compute_neprec(row["reference_text"], row["generated_text"]),
    axis=1
)

neprec_results = (
    df.groupby("model_id")["neprec"]
      .agg(["mean", "std"])
      .reset_index()
)

print(neprec_results)


         model_id      mean       std
0  gpt-4o-mini-v2  0.289932  0.254560
1       gpt-4o-v2  0.231257  0.185116


In [ ]:
!pip install openai
!pip install azure-identity
!pip install azure-core

from openai import AzureOpenAI

import os
from google.colab import userdata

endpoint = userdata.get('MODEL_ENDPOINT')
deployment = userdata.get('MODEL_DEPLOY')

subscription_key = userdata.get('MODEL_KEY')
api_version = userdata.get('MODEL_API_VERSION')

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

print("AzureOpenAI client defined.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.5/88.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.3/191.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.0/214.0 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 11.1 MB/s eta 0:00:00
AzureOpenAI client defined.


In [7]:
llm_prompt_content_template = """Você é um avaliador de texto inteligente. Sua tarefa é comparar duas listas de entidades extraídas de documentos legais: 'entidades de referência' e 'entidades geradas'.

As 'entidades de referência' representam as entidades corretas e esperadas.
As 'entidades geradas' são as entidades que foram extraídas por um modelo de linguagem.

Sua avaliação deve ser baseada na precisão semântica e contextual das 'entidades geradas' em relação às 'entidades de referência'. Ou seja, quão bem as entidades geradas correspondem, em significado e relevância para o contexto legal, às entidades de referência.

Retorne um único valor float entre 0.0 e 1.0, onde:
- 1.0 significa que todas as entidades geradas são semanticamente corretas e altamente relevantes em comparação com as de referência.
- 0.0 significa que nenhuma das entidades geradas é semanticamente correta ou relevante.
- Valores intermediários representam precisão parcial.

Considere sinônimos, termos relacionados e o contexto legal para determinar a precisão. Se uma entidade gerada é ligeiramente diferente, mas ainda assim semanticamente a mesma, considere-a parcialmente correta.

Entidades de Referência: {reference_entities}
Entidades Geradas: {generated_entities}

Retorne apenas o valor float, sem nenhum texto adicional. Por exemplo: 0.75
"""

print("Prompt content template for AzureOpenAI client defined.")

llm_coverage_prompt_template = """Você é um avaliador de texto inteligente. Sua tarefa é avaliar a 'cobertura' das 'entidades de referência' pelas 'entidades geradas'.
As 'entidades de referência' são as entidades esperadas e cruciais. As 'entidades geradas' são as entidades que foram extraídas por um modelo de linguagem, e tendem a ser mais abrangentes.

Você deve determinar qual proporção das 'entidades de referência' é semanticamente coberta ou 'contemplada' pelas 'entidades geradas'.
- Para cada entidade na lista de 'entidades de referência', verifique se existe uma entidade semanticamente equivalente ou um conceito relacionado na lista de 'entidades geradas'.
- Retorne um único valor float entre 0.0 e 1.0, onde:
- 1.0 significa que todas as entidades de referência são semanticamente cobertas pelas entidades geradas.
- 0.0 significa que nenhuma das entidades de referência é semanticamente coberta.
- Valores intermediários representam cobertura parcial.

Considere sinônimos, termos relacionados e o contexto legal para determinar a cobertura.

Entidades de Referência: {reference_entities}
Entidades Geradas: {generated_entities}

Retorne apenas o valor float, sem nenhum texto adicional. Por exemplo: 0.85
"""

print("Prompt content template for LLM coverage comparison defined.")

Prompt content template for AzureOpenAI client defined.
Prompt content template for LLM coverage comparison defined.


In [8]:
import re

def compute_llm_neprec_coverage(reference_entities: list, generated_entities: list) -> float:
    # Se a lista de entidades de referência estiver vazia, a cobertura é 1.0 (não há nada para cobrir)
    if not reference_entities:
        return 1.0

    # Se a lista de entidades geradas estiver vazia e há entidades de referência, a cobertura é 0.0
    if not generated_entities and reference_entities:
        return 0.0

    # Convert lists to string representations suitable for the prompt
    ref_entities_str = ", ".join(reference_entities)
    gen_entities_str = ", ".join(generated_entities)

    # Format the prompt using the global template
    prompt_content = llm_coverage_prompt_template.format(
        reference_entities=ref_entities_str,
        generated_entities=gen_entities_str
    )

    try:
        # Invoque o LLM com o prompt formatado usando AzureOpenAI client
        response = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are an intelligent text evaluator. Always respond with a float value."
                },
                {
                    "role": "user",
                    "content": prompt_content
                }
            ],
            max_completion_tokens=50,
            temperature=0.0,
            model=deployment
        )
        llm_response = response.choices[0].message.content

        # Processar a resposta do LLM, extraindo o valor float retornado
        match = re.search(r"\d+\.\d+", llm_response)
        if match:
            score = float(match.group(0))
            # Garantir que o score esteja dentro do range válido [0.0, 1.0]
            return max(0.0, min(1.0, score))
        else:
            print(f"Warning: LLM did not return a valid float for coverage. Response: {llm_response}")
            return 0.0 # Default para 0 se o parsing falhar

    except Exception as e:
        print(f"Error during LLM coverage invocation: {e}")
        return 0.0 # Retorna 0 em caso de erro

print("Function `compute_llm_neprec_coverage` defined.")

Function `compute_llm_neprec_coverage` defined.


In [9]:
import re

def compute_llm_neprec_smart_compare(reference_entities: list, generated_entities: list) -> float:
    # Se a lista generated_entities estiver vazia, retorne 1.0
    if not generated_entities:
        return 1.0

    # Utilizar o PromptTemplate definido para formatar as listas de entidades
    # Convert lists to string representations suitable for the prompt
    ref_entities_str = ", ".join(reference_entities)
    gen_entities_str = ", ".join(generated_entities)

    # Format the prompt using the global template
    prompt_content = llm_prompt_content_template.format(
        reference_entities=ref_entities_str,
        generated_entities=gen_entities_str
    )

    try:
        # Invoque o LLM com o prompt formatado usando AzureOpenAI client
        response = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are an intelligent text evaluator. Always respond with a float value.",
                },
                {
                    "role": "user",
                    "content": prompt_content,
                }
            ],
            max_completion_tokens=50, # Limiting tokens as only a float is expected
            temperature=0.0,
            model=deployment # Use the deployment model from the client setup
        )
        llm_response = response.choices[0].message.content

        # Processe a resposta do LLM, extraindo o valor float retornado
        # Use regex to find a float value in the response
        match = re.search(r"\d+\.\d+", llm_response)
        if match:
            score = float(match.group(0))
            # Ensure the score is within the valid range [0.0, 1.0]
            return max(0.0, min(1.0, score))
        else:
            print(f"Warning: LLM did not return a valid float. Response: {llm_response}")
            return 0.0 # Default to 0 if parsing fails

    except Exception as e:
        print(f"Error during LLM invocation: {e}")
        return 0.0 # Return 0 in case of an error

print("Function `compute_llm_neprec_smart_compare` defined.")

Function `compute_llm_neprec_smart_compare` defined.


In [10]:
import os
import ast # Required for safer parsing of LLM output

llm_extract_prompt_template = """Você é um assistente de IA especialista em extração de entidades legais de texto em Português do Brasil.
Sua tarefa é identificar e listar todas as entidades legais relevantes no texto fornecido.
Considere as seguintes categorias de entidades legais:
- PER (Pessoas, nomes de indivíduos)
- ORG (Órgãos, empresas, tribunais, organizações)
- LOC (Localidades, endereços, cidades, países)
- DATE (Datas, períodos)
- MONEY (Valores monetários)

Liste as entidades no formato de lista Python de strings, onde cada string é a entidade encontrada. Garanta que cada entidade esteja em minúsculas e sem espaços extras.
Se nenhuma entidade for encontrada, retorne uma lista vazia `[]`.

Texto: {text}

Entidades extraídas (apenas a lista Python):"""

# No llm_extract_chain needed for direct AzureOpenAI client calls

def extract_entities_llm(text: str) -> list:
    try:
        # Format the prompt for the AzureOpenAI client
        prompt_content = llm_extract_prompt_template.format(text=text)

        # Invoke the LLM with the formatted prompt using AzureOpenAI client
        response = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are an AI assistant for legal entity extraction. Always respond with a Python list of strings, e.g., ['entity 1', 'entity 2'].",
                },
                {
                    "role": "user",
                    "content": prompt_content,
                }
            ],
            max_completion_tokens=1000, # Increased tokens for extraction
            temperature=0.0,
            model=deployment # Use the deployment model from the client setup
        )
        llm_response = response.choices[0].message.content

        # Attempt to parse the response as a Python list
        # Using ast.literal_eval for safer parsing than eval()
        entities = ast.literal_eval(llm_response)
        if isinstance(entities, list):
            # Convert all entities to lowercase for consistency
            return [str(entity).strip().lower() for entity in entities]
        else:
            print(f"Warning: LLM did not return a valid list for extraction. Response: {llm_response}")
            return []
    except (SyntaxError, ValueError) as e:
        print(f"Error parsing LLM response as a list: {e}. Response: {llm_response}")
        return []
    except Exception as e:
        print(f"Error during LLM entity extraction: {e}")
        return []

print("Function `extract_entities_llm` and its prompt defined.")

Function `extract_entities_llm` and its prompt defined.


In [11]:
import re

def compute_llm_neprec_coverage_llm_extracted_entities(reference_text: str, generated_text: str) -> float:
    # 1. Extrair entidades do texto de referência usando o LLM
    ref_entities_llm = extract_entities_llm(reference_text)

    # 2. Extrair entidades do texto gerado usando o LLM
    gen_entities_llm = extract_entities_llm(generated_text)

    # 3. Calcular a cobertura usando a função de cobertura com LLM-extracted entities
    # A função compute_llm_neprec_coverage já lida com listas vazias de generated_entities
    llm_coverage_score = compute_llm_neprec_coverage(ref_entities_llm, gen_entities_llm)

    return llm_coverage_score

print("Function `compute_llm_neprec_coverage_llm_extracted_entities` defined.")

Function `compute_llm_neprec_coverage_llm_extracted_entities` defined.


In [12]:
def compute_llm_neprec_extract_and_compare(reference_text: str, generated_text: str) -> float:
    # 1. Extrair entidades do texto de referência usando o LLM
    ref_entities_llm = extract_entities_llm(reference_text)

    # 2. Extrair entidades do texto gerado usando o LLM
    gen_entities_llm = extract_entities_llm(generated_text)

    # 3. Calcular a precisão semântica usando a função de comparação com LLM
    # A função compute_llm_neprec_smart_compare já lida com listas vazias de generated_entities
    llm_smart_compare_score = compute_llm_neprec_smart_compare(ref_entities_llm, gen_entities_llm)

    return llm_smart_compare_score

print("Function `compute_llm_neprec_extract_and_compare` defined.")

Function `compute_llm_neprec_extract_and_compare` defined.


In [13]:
llm_validation_results_full_df = []

print(f"Starting LLM-based NePREC validations for the full dataset ({len(df)} rows). This may take some time...")

for index, row in df.iterrows():
    ref_text = row["reference_text"]
    gen_text = row["generated_text"]
    original_neprec = row["neprec"]
    model_id = row["model_id"]

    # 1. Calculate LLM NePREC (semantic comparison of spaCy extracted entities)
    ref_entities_spacy = extract_entities(ref_text)
    gen_entities_spacy = extract_entities(gen_text)

    if not gen_entities_spacy:
        llm_smart_compare_neprec = 1.0 # No hallucination if no entities were generated by spaCy
    else:
        llm_smart_compare_neprec = compute_llm_neprec_smart_compare(ref_entities_spacy, gen_entities_spacy)

    # 2. Calculate LLM NePREC (LLM extraction and semantic comparison)
    llm_extract_and_compare_neprec = compute_llm_neprec_extract_and_compare(ref_text, gen_text)

    # 3. Calculate LLM NePREC Coverage (using spaCy entities for comparison)
    llm_coverage_spacy_entities_neprec = compute_llm_neprec_coverage(ref_entities_spacy, gen_entities_spacy)

    # 4. Calculate LLM NePREC Coverage (using LLM extracted entities for comparison)
    llm_coverage_llm_entities_neprec = compute_llm_neprec_coverage_llm_extracted_entities(ref_text, gen_text)

    llm_validation_results_full_df.append({
        "index": index,
        "model_id": model_id,
        "original_neprec": original_neprec,
        "llm_smart_compare_neprec": llm_smart_compare_neprec,
        "llm_extract_and_compare_neprec": llm_extract_and_compare_neprec,
        "llm_coverage_spacy_entities_neprec": llm_coverage_spacy_entities_neprec,
        "llm_coverage_llm_entities_neprec": llm_coverage_llm_entities_neprec
    })

    # Optional: Print progress for long running tasks
    if (index + 1) % 50 == 0 or (index + 1) == len(df):
        print(f"Processed {index + 1}/{len(df)} rows.")

print("\n--- Finished processing full dataset ---")

# Convert results to DataFrame
full_df_llm_metrics = pd.DataFrame(llm_validation_results_full_df)
print("\nDataFrame `full_df_llm_metrics` created with all NePREC scores.")
print(full_df_llm_metrics.head())

Starting LLM-based NePREC validations for the full dataset (210 rows). This may take some time...
Processed 50/210 rows.
Processed 100/210 rows.
Processed 150/210 rows.
Processed 200/210 rows.
Processed 210/210 rows.

--- Finished processing full dataset ---

DataFrame `full_df_llm_metrics` created with all NePREC scores.
   index   model_id  original_neprec  llm_smart_compare_neprec  \
0      0  gpt-4o-v2         0.083333                      0.30   
1      1  gpt-4o-v2         0.285714                      0.50   
2      2  gpt-4o-v2         0.142857                      0.30   
3      3  gpt-4o-v2         0.375000                      0.55   
4      4  gpt-4o-v2         0.833333                      0.65   

   llm_extract_and_compare_neprec  llm_coverage_spacy_entities_neprec  \
0                            0.60                            0.222222   
1                            0.75                            1.000000   
2                            0.70                           

In [14]:
print(full_df_llm_metrics.groupby("model_id")[[
    "original_neprec",
    "llm_smart_compare_neprec",
    "llm_extract_and_compare_neprec",
    "llm_coverage_spacy_entities_neprec",
    "llm_coverage_llm_entities_neprec"
]].agg(['mean', 'std', 'min', 'max']).to_string())


               original_neprec                          llm_smart_compare_neprec                      llm_extract_and_compare_neprec                     llm_coverage_spacy_entities_neprec                     llm_coverage_llm_entities_neprec                     
                          mean       std  min       max                     mean       std  min   max                           mean       std  min  max                               mean       std  min  max                             mean       std   min  max
model_id                                                                                                                                                                                                                                                             
gpt-4o-mini-v2        0.289932  0.254560  0.0  1.000000                 0.403524  0.190959  0.0  0.75                       0.594762  0.182125  0.2  1.0                           0.405733  0.247466  0.0  1.0       